# 用户和活动关联关系处理


整个数据集中活动数目（events.csv）太多，所以下面的处理我们找出只在训练集和测试集中出现的活动和用户集合，并对他们重新编制索引

In [1]:
#保存数据
import cPickle

import itertools

#处理事件字符串
import datetime

import numpy as np
import scipy.io as sio
import scipy.sparse as ss

#相似度/距离
import scipy.spatial.distance as ssd

from collections import defaultdict
from sklearn.preprocessing import normalize

In [3]:
"""
我们只关心train和test中出现的user和event，因此重点处理这部分关联数据

train.csv 有6列：
user：用户ID
event：活动ID
invited：是否被邀请（0/1）
timestamp：ISO-8601 UTC格式时间字符串，表示用户看到该活动的时间
interested, and not_interested

Test.csv 除了没有interested, and not_interested，其余列与train相同
 """
# 数据地址
dpath = 'E:/AI/00/data/EventRecommendation/'


# 统计训练集中有多少不同的用户的events
uniqueUsers = set()
uniqueEvents = set()

#倒排表
#统计每个用户参加的活动   / 每个活动参加的用户
eventsForUser = defaultdict(set)
usersForEvent = defaultdict(set)
    
for filename in ["train.csv", "test.csv"]:
    f = open(dpath+filename, 'rb')
    
    #忽略第一行（列名字）
    f.readline().strip().split(",")
    
    for line in f:    #对每条记录
        cols = line.strip().split(",")
        uniqueUsers.add(cols[0])   #第一列为用户ID
        uniqueEvents.add(cols[1])   #第二列为活动ID
        
        #eventsForUser[cols[0]].add(cols[1])    #该用户参加了这个活动
        #usersForEvent[cols[1]].add(cols[0])    #该活动被用户参加
    f.close()


n_uniqueUsers = len(uniqueUsers)
n_uniqueEvents = len(uniqueEvents)

print("number of uniqueUsers :%d" % n_uniqueUsers)
print("number of uniqueEvents :%d" % n_uniqueEvents)


number of uniqueUsers :3391
number of uniqueEvents :13418


In [4]:

#用户关系矩阵表，可用于后续LFM/SVD++处理的输入
#这是一个稀疏矩阵，记录用户对活动感兴趣
userEventScores = ss.dok_matrix((n_uniqueUsers, n_uniqueEvents))
userIndex = dict()
eventIndex = dict()

#重新编码用户索引字典
for i, u in enumerate(uniqueUsers):
    userIndex[u] = i
    
#重新编码活动索引字典    
for i, e in enumerate(uniqueEvents):
    eventIndex[e] = i

n_records = 0
ftrain = open(dpath+"train.csv", 'rb')
ftrain.readline()
for line in ftrain:
    cols = line.strip().split(",")
    i = userIndex[cols[0]]  #用户
    j = eventIndex[cols[1]] #活动
    
    eventsForUser[i].add(j)    #该用户参加了这个活动
    usersForEvent[j].add(i)    #该活动被用户参加
        
    #userEventScores[i, j] = int(cols[4]) - int(cols[5])   #interested - not_interested
    score = int(cols[4])
    #if score == 0:  #0在稀疏矩阵中表示该元素不存在，因此借用-1表示interested=0
    #userEventScores[i, j] = -1
    #else:
    userEventScores[i, j] = score
ftrain.close()


In [5]:
 
##统计每个用户参加的活动，后续用于将用户朋友参加的活动影响到用户
cPickle.dump(eventsForUser, open("PE_eventsForUser.pkl", 'wb'))
##统计活动参加的用户
cPickle.dump(usersForEvent, open("PE_usersForEvent.pkl", 'wb'))

#保存用户-活动关系矩阵R，以备后用
sio.mmwrite("PE_userEventScores", userEventScores)

In [6]:
#保存用户索引表
cPickle.dump(userIndex, open("PE_userIndex.pkl", 'wb'))
#保存活动索引表
cPickle.dump(eventIndex, open("PE_eventIndex.pkl", 'wb'))


In [7]:
# 为了防止不必要的计算，我们找出来所有关联的用户 或者 关联的event
# 所谓的关联用户，指的是至少在同一个event上有行为的用户pair
# 关联的event指的是至少同一个user有行为的event pair
uniqueUserPairs = set()
uniqueEventPairs = set()
for event in uniqueEvents:
    i = eventIndex[event]
    users = usersForEvent[i]
    if len(users) > 2:
        uniqueUserPairs.update(itertools.combinations(users, 2))
        
for user in uniqueUsers:
    u = userIndex[user]
    events = eventsForUser[u]
    if len(events) > 2:
        uniqueEventPairs.update(itertools.combinations(events, 2))

In [8]:
cPickle.dump(uniqueUserPairs, open("FE_uniqueUserPairs.pkl", 'wb'))
cPickle.dump(uniqueEventPairs, open("PE_uniqueEventPairs.pkl", 'wb'))

In [9]:
#训练集和测试集中出现的用户数目和事件数目远小于users.csv出现的用户数和events.csv出现的事件数

In [11]:
len(uniqueEventPairs)

57824

In [12]:
len(uniqueUserPairs)

84051

In [13]:
uniqueUserPairs

{(3107, 2511),
 (814, 2929),
 (231, 1333),
 (1109, 2831),
 (82, 2496),
 (151, 3004),
 (438, 2511),
 (1806, 1970),
 (3073, 942),
 (580, 2375),
 (838, 2590),
 (767, 365),
 (2060, 1567),
 (523, 2017),
 (1955, 3068),
 (101, 2293),
 (175, 1362),
 (1139, 45),
 (1550, 2659),
 (2344, 966),
 (3335, 982),
 (1115, 365),
 (1702, 2459),
 (3182, 1513),
 (23, 1970),
 (2224, 556),
 (122, 635),
 (2256, 1087),
 (32, 367),
 (1173, 2473),
 (1979, 2547),
 (701, 2431),
 (2825, 916),
 (772, 379),
 (403, 1995),
 (1986, 671),
 (2424, 483),
 (3259, 3089),
 (1676, 3040),
 (1301, 2861),
 (834, 3089),
 (2768, 2708),
 (2447, 883),
 (105, 760),
 (2798, 1526),
 (1866, 2531),
 (626, 2565),
 (2611, 3326),
 (2956, 2580),
 (3380, 995),
 (1560, 2897),
 (672, 174),
 (169, 1379),
 (2351, 2879),
 (1061, 1272),
 (819, 1733),
 (3079, 1205),
 (1702, 1733),
 (3179, 91),
 (733, 3363),
 (175, 1143),
 (669, 783),
 (197, 2530),
 (3331, 206),
 (544, 2748),
 (2486, 3013),
 (3280, 1254),
 (3248, 720),
 (3232, 1205),
 (1415, 297),
 (110